¡Hola!

Mi nombre es Marcos Torres y tengo el gusto de revisar tu código el día de hoy.

Cuando vea algo notable o algún asunto en el notebook, te dejaré un comentario o un hint. Se que encontraras la mejor respuesta para resolver todos los comentarios, de no ser así, no te preocupes en futuras iteraciones dejaré comentarios y pistas más específicos.

Este proceso es muy parecido al que se recibe de un gerente o de un Senior Data Scientist en un trabajo real, por lo que te estarás preparando para la experiencia en la vida real.

Encontrarás comentarios en verde, amarillo o rojo como los siguientes:

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Buen trabajo. ¡Lo hiciste muy bien!
</div>

<div class="alert alert-block alert-warning">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Nota. Se puede mejorar.
</div>

<div class="alert alert-block alert-danger">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Necesitas corregirlo. Este bloque indica que se requiere una correción. El trabajo no se acepta si tiene estos bloques.
</div>

Puedes responder a mis comentarios usando estos bloques:

<div class="alert alert-block alert-info">
<b>Respuesta del estudiante.</b> <a class="tocSkip"></a>
</div>

# Descripción

La compañía de seguros Sure Tomorrow quiere resolver varias tareas con la ayuda de machine learning y te pide que evalúes esa posibilidad.
- Tarea 1: encontrar clientes que sean similares a un cliente determinado. Esto ayudará a los agentes de la compañía con el marketing.
- Tarea 2: predecir la probabilidad de que un nuevo cliente reciba una prestación del seguro. ¿Puede un modelo de predictivo funcionar mejor que un modelo dummy?
- Tarea 3: predecir el número de prestaciones de seguro que un nuevo cliente pueda recibir utilizando un modelo de regresión lineal.
- Tarea 4: proteger los datos personales de los clientes sin afectar al modelo del ejercicio anterior. Es necesario desarrollar un algoritmo de transformación de datos que dificulte la recuperación de la información personal si los datos caen en manos equivocadas. Esto se denomina enmascaramiento u ofuscación de datos. Pero los datos deben protegerse de tal manera que no se vea afectada la calidad de los modelos de machine learning. No es necesario elegir el mejor modelo, basta con demostrar que el algoritmo funciona correctamente.


# Preprocesamiento y exploración de datos

## Inicialización

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

import seaborn as sns

import sklearn.linear_model
import sklearn.metrics
from sklearn.neighbors import (NearestNeighbors, KNeighborsClassifier)
from sklearn.preprocessing import MaxAbsScaler

from sklearn.model_selection import train_test_split

from IPython.display import display

from scipy.spatial import distance

import warnings

In [ ]:
warnings.simplefilter('ignore')

## Carga de datos

Carga los datos y haz una revisión básica para comprobar que no hay problemas obvios.

In [ ]:
try:
    df = pd.read_csv('insurance_us.csv')
except:
    df = pd.read_csv('/datasets/insurance_us.csv')

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Bien, usaste una celda independiente para importar las librerías y otra para leer los datos.
</div>


In [ ]:
df.head()

Renombramos las columnas para que el código se vea más coherente con su estilo.

In [ ]:
df = df.rename(columns={'Gender': 'gender', 'Age': 'age', 'Salary': 'income', 'Family members': 'family_members', 'Insurance benefits': 'insurance_benefits'})

In [ ]:
df.sample(10)

In [ ]:
df.info()

In [ ]:
#Se cambia el tipo de datos de la columna age
df['age'] = df['age'].astype('int64')

In [ ]:
df.info()

In [ ]:
df.describe()

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Buen uso de los métodos de pandas para explorar los datos.
</div>

Los datos que se presentan en el dataset tienen consistencia, es decir no se ven valores raros, no se encuentran valores ausentes

In [ ]:
columns = ['gender', 'age', 'income', 'family_members', 'insurance_benefits']

In [ ]:
for column in columns:
    df[column].hist(figsize=(6,4))
    plt.title(column)
    plt.show()

De los histrogramas mostrados arriba vemos que hay igualdad en los datos, la mayor cantidad de personas se encuentran entre los 25 y los 40 años de edad, con un ingreso de 35,000 a 50,000 dólares.
Estas personas tienen entre 0 y 2 miembros de familia, y la mayoría no ha tenido que utilizar el seguro.

<div class="alert alert-block alert-warning">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Bien, la exploración con los histogramas es correcta. Tal vez incluiría título en cada gráfica en caso de que se quiera usar de forma independiente y en la de gender falta indicar que significa 0 y 1.
</div>

<div class="alert alert-block alert-info">
<b>Respuesta del estudiante.</b> <a class="tocSkip"></a>
    <br>Al generar las gráficas por medio de un for, no encontré forma de adicionar la leyenda al gender
</div>

## Análisis exploratorio de datos

Vamos a comprobar rápidamente si existen determinados grupos de clientes observando el gráfico de pares.

In [ ]:
g = sns.pairplot(df, kind='hist')
g.fig.set_size_inches(12, 12)

De acuerdo, es un poco complicado detectar grupos obvios (clústeres) ya que es difícil combinar diversas variables simultáneamente (para analizar distribuciones multivariadas). Ahí es donde LA y ML pueden ser bastante útiles.

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Buen uso de la gráfica de pares.
</div>

# Tarea 1. Clientes similares

En el lenguaje de ML, es necesario desarrollar un procedimiento que devuelva los k vecinos más cercanos (objetos) para un objeto dado basándose en la distancia entre los objetos.
Es posible que quieras revisar las siguientes lecciones (capítulo -> lección)
- Distancia entre vectores -> Distancia euclidiana
- Distancia entre vectores -> Distancia Manhattan

Para resolver la tarea, podemos probar diferentes métricas de distancia.

Escribe una función que devuelva los k vecinos más cercanos para un $n^{th}$ objeto basándose en una métrica de distancia especificada. A la hora de realizar esta tarea no debe tenerse en cuenta el número de prestaciones de seguro recibidas.
Puedes utilizar una implementación ya existente del algoritmo kNN de scikit-learn (consulta [el enlace](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)) o tu propia implementación.
Pruébalo para cuatro combinaciones de dos casos- Escalado
  - los datos no están escalados
  - los datos se escalan con el escalador [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html)
- Métricas de distancia
  - Euclidiana
  - Manhattan

Responde a estas preguntas:- ¿El hecho de que los datos no estén escalados afecta al algoritmo kNN? Si es así, ¿cómo se manifiesta?- ¿Qué tan similares son los resultados al utilizar la métrica de distancia Manhattan (independientemente del escalado)?

In [ ]:
feature_names = ['gender', 'age', 'income', 'family_members']

In [ ]:
def get_knn(df, n, k, metric):
    
    """
    Devuelve los k vecinos más cercanos

    :param df: DataFrame de pandas utilizado para encontrar objetos similares dentro del mismo lugar    
    :param n: número de objetos para los que se buscan los vecinos más cercanos    
    :param k: número de vecinos más cercanos a devolver
    :param métrica: nombre de la métrica de distancia    """

    nbrs = NearestNeighbors(n_neighbors=k, metric=metric).fit(df[feature_names])
    nbrs_distances, nbrs_indices = nbrs.kneighbors([df.iloc[n][feature_names]], k, return_distance=True)
    
    df_res = pd.concat([
        df.iloc[nbrs_indices[0]], 
        pd.DataFrame(nbrs_distances.T, index=nbrs_indices[0], columns=['distance'])
        ], axis=1)
    
    return df_res

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

¡Muy bien! La función está bien definida y es una muy buena práctica incluir la sección de comentarios del inicio que explica los parámetros, eso le da un toque más profesional.
</div>

Escalar datos.

In [ ]:
feature_names = ['gender', 'age', 'income', 'family_members']

transformer_mas = MaxAbsScaler().fit(df[feature_names].to_numpy())

df_scaled = df.copy()
df_scaled.loc[:, feature_names] = transformer_mas.transform(df[feature_names].to_numpy())

In [ ]:
df_scaled.sample(5)

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Se escalaron correctamente los datos. Y buen uso del método copy para crear un data frame nuevo sin modificar el original.
</div>

Ahora, vamos a obtener registros similares para uno determinado, para cada combinación

In [ ]:
get_knn(df_scaled,25,5,'euclidean')

In [ ]:
#Se tomarán 10 indices aleatorios de los datos escalados y sin escalara para comprobar si se afectan los resultados.

indices = df.sample(10).index
indices

In [ ]:
for index in indices:
    mean_distance = get_knn(df, index, 10, 'euclidean')['distance'].mean()
    print('La distancia promedio de los elementos sin escalar es:', mean_distance)

In [ ]:
for index in indices:
    mean_distance_scale = get_knn(df_scaled, index, 10, 'euclidean')['distance'].mean()
    print('La distancia promedio de los elementos escalados es: ', mean_distance_scale)

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Buen uso de los prints en el ciclo for.
</div>

In [ ]:
df.loc[[indices[0], get_knn(df, indices[0], 2, 'euclidean').iloc[1].name]]

In [ ]:
distance.euclidean(df.loc[indices[0]].values, df.loc[get_knn(df, indices[0], 2, 'euclidean').iloc[1].name].values)

In [ ]:
df_scaled.loc[[indices[0], get_knn(df_scaled, indices[0], 2, 'euclidean').iloc[1].name]]

In [ ]:
distance.euclidean(df_scaled.loc[indices[0]].values, 
                   df_scaled.loc[get_knn(df_scaled, indices[0], 2, 'euclidean').iloc[1].name].values)

In [ ]:
nneighbor = {}

for index in indices:
    nneighbor[index] = get_knn(df, index, 10, 'euclidean').index


In [ ]:
distances = []

for index, value in nneighbor.items():
    distances.append([distance.euclidean(df.loc[index], df.loc[index_v]) for index_v in value])
    
distances = np.array(distances)
dist_df = pd.DataFrame(np.transpose(distances), columns=nneighbor.keys())
dist_df.mean()

In [ ]:
index_euclidean = {}
index_manhattan = {}

for index in indices:
    index_euclidean[index] = get_knn(df, index, 10, 'euclidean').index
    index_manhattan[index] = get_knn(df, index, 10, 'cityblock').index
euclidean = pd.DataFrame(index_euclidean)
manhattan = pd.DataFrame(index_manhattan)

In [ ]:
euclidean

In [ ]:
manhattan

In [ ]:
def diferencias_dist(list_1, list_2):
    count = 0
    for item in list_1:
        if item not in list_2:
            count += 1
    return count

In [ ]:
count = []
for index in indices:
    count.append(diferencias_dist(index_euclidean[index], index_manhattan[index]))
    print(f'Diferentes vecinos entre distancia euclideana y manhattan con el indice {index} es {diferencias_dist(index_euclidean[index], index_manhattan[index])} vecinos diferentes')
print()
print(f'Porcentaje de vecinos diferentes: {sum(count)/euclidean.values.size*100} %')

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Bien exploraste todos los casos usando el data frame y el ciclo for.
</div>

Respuestas a las preguntas

**¿El hecho de que los datos no estén escalados afecta al algoritmo kNN? Si es así, ¿cómo se manifiesta?** 

Si, el algoritmo kNN se ve afectado por la diferencia en los valores de la columna `income`, y utilizando los valores escalados las diferencias son menores, o son casi similares por lo que las columnas de características son evaluadas al mismo tiempo y se encuentran los vecinos adecuados con todas las características y no solo con los que se tienen el ingreso igual.
Como se muestra arriba la distancia de los vecinos más cercanos es menor con los datos escalados contra los datos sin escalar

**¿Qué tan similares son los resultados al utilizar la métrica de distancia Manhattan (independientemente del escalado)?** 

La diferencia entre los vecinos más cercanos entre las distancias euclidianas y manhattan es del 4%, es decir de cada 100 vecinos solo 4 son diferentes por lo que podemos decir que las distancias son iguales.

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Buenos comentarios.
</div>

# Tarea 2. ¿Es probable que el cliente reciba una prestación del seguro?

En términos de machine learning podemos considerarlo como una tarea de clasificación binaria.

Con el valor de `insurance_benefits` superior a cero como objetivo, evalúa si el enfoque de clasificación kNN puede funcionar mejor que el modelo dummy.
Instrucciones:
- Construye un clasificador basado en KNN y mide su calidad con la métrica F1 para k=1...10 tanto para los datos originales como para los escalados. Sería interesante observar cómo k puede influir en la métrica de evaluación y si el escalado de los datos provoca alguna diferencia. Puedes utilizar una implementación ya existente del algoritmo de clasificación kNN de scikit-learn (consulta [el enlace](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)) o tu propia implementación.
- Construye un modelo dummy que, en este caso, es simplemente un modelo aleatorio. Debería devolver "1" con cierta probabilidad. Probemos el modelo con cuatro valores de probabilidad: 0, la probabilidad de pagar cualquier prestación del seguro, 0.5, 1.
La probabilidad de pagar cualquier prestación del seguro puede definirse como
$$
P\{\text{prestación de seguro recibida}\}=\frac{\text{número de clientes que han recibido alguna prestación de seguro}}{\text{número total de clientes}}.
$$

Divide todos los datos correspondientes a las etapas de entrenamiento/prueba respetando la proporción 70:30.

In [ ]:
df['insurance_benefits_received'] = df['insurance_benefits'] > 0
df['insurance_benefits_received'] = df['insurance_benefits_received'].astype('int64')

In [ ]:
df['insurance_benefits_received'].value_counts()

Hay un gran desequilibrio de clases mostrando la mayoría de datos cargados hacia quienes no han recibido beneficios del seguro.

Se dividirá el dataframe en características y objetivos

In [ ]:
X = df.drop(['insurance_benefits', 'insurance_benefits_received'], axis=1)
y = df['insurance_benefits_received']

In [ ]:
print(X.shape)
print(y.shape)

Se crearan los conjuntos de prueba y entrenamiento

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=12345)

print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Buen uso de la función train_test_split.
</div>

Se escalarán los conjuntos de datos de las características para poder realizar los cálculos y comparar resultados con valores escalados y sin escalar

In [ ]:
transformer_mas = MaxAbsScaler().fit(X_train)

X_train_scaled = pd.DataFrame(transformer_mas.transform(X_train), columns = X_train.columns, index = X_train.index)
X_test_scaled = pd.DataFrame(transformer_mas.transform(X_test), columns =  X_test.columns, index = X_test.index)

In [ ]:
X_train_scaled.head()

In [ ]:
X_test_scaled.head()

Se crea la función para evaluar el modelo de predicción

In [ ]:
def eval_classifier(y_true, y_pred):
    
    f1_score = sklearn.metrics.f1_score(y_true, y_pred)
    
    cm = sklearn.metrics.confusion_matrix(y_true, y_pred, normalize='all')
    plt.figure(figsize=(8,6), dpi=100)
    sns.set(font_scale= 1.1)
    ax = sns.heatmap(cm, annot=True)
    ax.xaxis.set_ticklabels(['Negative', 'Positive'])
    ax.yaxis.set_ticklabels(['Negative', 'Positive'])
    ax.set_title(f'Confusion Matrix for F1 = {f1_score}')
    plt.show()
    
    return f1_score, cm

<div class="alert alert-block alert-warning">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Bien definiste correctamente la función para encontrar el f1 score y la matriz de confusión. Otra versión de la matriz de confusión que aporta más claridad es cuando se normaliza por la predicción o por los valores reales, de esta forma en cada columna (o renglón) suma 1 y es más fácil visualizar la clasificación correcta de cada categoría.
</div>

<div class="alert alert-block alert-info">
<b>Respuesta del estudiante.</b> <a class="tocSkip"></a>
    <br>No entendí el comentario o como hacer esto
</div>

<div class="alert alert-block alert-warning">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Esto se consigue usando los valores 'true' y 'pred' en normalize. https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
</div>

In [ ]:
results_nsc = []

for k in range(10):
    knn_model = KNeighborsClassifier(n_neighbors= k+1)
    knn_model.fit(X_train, y_train)
    predictions = knn_model.predict(X_test)
    results_nsc.append(eval_classifier(y_test, predictions)[0])
    

In [ ]:
results_sc = []

for k in range(10):
    knn_model = KNeighborsClassifier(n_neighbors= k+1)
    knn_model.fit(X_train_scaled, y_train)
    predictions = knn_model.predict(X_test_scaled)
    
    results_sc.append(eval_classifier(y_test, predictions)[0])
results_sc

<div class="alert alert-block alert-warning">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Bien, de manera alternativa a mostrar solo los números en seaborn hay opciones para visualizar matrices de confusión.
</div>

In [ ]:
result = pd.DataFrame()
result['non_scaled'] = results_nsc
result['scaled'] = results_sc
result.plot(kind='line', figsize=(9,6), title='F1 Score for neighbors', grid=True)
plt.show()

In [ ]:
def rnd_model_predict(P, size, seed=42):

    rng = np.random.default_rng(seed=seed)
    return rng.binomial(n=1, p=P, size=size)

In [ ]:
for P in [0, df['insurance_benefits_received'].sum() / len(df), 0.5, 1]:

    print(f'La probabilidad: {P:.2f}')
    y_pred_rnd = rnd_model_predict(P, df.shape[0]) 
        
    eval_classifier(df['insurance_benefits_received'], y_pred_rnd)
    
    print()

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Muy bien.
</div>

# Tarea 3. Regresión (con regresión lineal)

Con `insurance_benefits` como objetivo, evalúa cuál sería la RECM de un modelo de regresión lineal.

Construye tu propia implementación de regresión lineal. Para ello, recuerda cómo está formulada la solución de la tarea de regresión lineal en términos de LA. Comprueba la RECM tanto para los datos originales como para los escalados. ¿Puedes ver alguna diferencia en la RECM con respecto a estos dos casos?

Denotemos- $X$: matriz de características; cada fila es un caso, cada columna es una característica, la primera columna está formada por unidades- $y$ — objetivo (un vector)- $\hat{y}$ — objetivo estimado (un vector)- $w$ — vector de pesos
La tarea de regresión lineal en el lenguaje de las matrices puede formularse así:
$$
y = Xw
$$

El objetivo de entrenamiento es entonces encontrar esa $w$ w que minimice la distancia L2 (ECM) entre $Xw$ y $y$:

$$
\min_w d_2(Xw, y) \quad \text{or} \quad \min_w \text{MSE}(Xw, y)
$$

Parece que hay una solución analítica para lo anteriormente expuesto:
$$
w = (X^T X)^{-1} X^T y
$$

La fórmula anterior puede servir para encontrar los pesos $w$ y estos últimos pueden utilizarse para calcular los valores predichos
$$
\hat{y} = X_{val}w
$$

Divide todos los datos correspondientes a las etapas de entrenamiento/prueba respetando la proporción 70:30. Utiliza la métrica RECM para evaluar el modelo.

In [ ]:
class MyLinearRegression:
    
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [ ]:
def eval_regressor(y_true, y_pred):
    try:
        rmse = math.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
        print(f'RMSE: {rmse:.2f}')
    
        r2_score = math.sqrt(sklearn.metrics.r2_score(y_true, y_pred))
        print(f'R2: {r2_score:.2f}')    
    except:
        print('Demonidador valor 0')

In [ ]:
X = df[['age', 'gender', 'income', 'family_members']].to_numpy()
y = df['insurance_benefits'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12345)

lr = MyLinearRegression()

lr.fit(X_train, y_train)
print(lr.w)

y_test_pred = lr.predict(X_test)
eval_regressor(y_test, y_test_pred)

In [ ]:
lr = MyLinearRegression()

lr.fit(X_train_scaled, y_train)

y_test_pred = lr.predict(X_test_scaled)
print(lr.w)
eval_regressor(y_test, y_test_pred)

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Se entreno y evaluó correctamente el modelo de regresión lineal.
</div>

# Tarea 4. Ofuscar datos

Lo mejor es ofuscar los datos multiplicando las características numéricas (recuerda que se pueden ver como la matriz $X$) por una matriz invertible $P$. 

$$
X' = X \times P
$$

Trata de hacerlo y comprueba cómo quedarán los valores de las características después de la transformación. Por cierto, la propiedad de invertibilidad es importante aquí, así que asegúrate de que $P$ sea realmente invertible.

Puedes revisar la lección 'Matrices y operaciones matriciales -> Multiplicación de matrices' para recordar la regla de multiplicación de matrices y su implementación con NumPy.

In [ ]:
personal_info_column_list = ['gender', 'age', 'income', 'family_members']
df_pn = df[personal_info_column_list]

In [ ]:
X = df_pn.to_numpy()
X

Generar una matriz aleatoria $P$.

In [ ]:
rng = np.random.default_rng(seed=42)
P = rng.random(size=(X.shape[1], X.shape[1]))

Comprobar que la matriz P sea invertible

In [ ]:
P.dot(np.linalg.inv(P)).round(14)

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Se verificó correctamente que la matriz sea invertible.
</div>

¿Puedes adivinar la edad o los ingresos de los clientes después de la transformación?

In [ ]:
X_ofus = X.dot(P)
X_ofus

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Se calculó correctamente.
</div>

Conociendo la matriz P con sus valores se pueden conocer los valores iniciales de la matriz ofuscada

¿Puedes recuperar los datos originales de $X'$ si conoces $P$? Intenta comprobarlo a través de los cálculos moviendo $P$ del lado derecho de la fórmula anterior al izquierdo. En este caso las reglas de la multiplicación matricial son realmente útiles

In [ ]:
X_ofus.dot(np.linalg.inv(P))

Muestra los tres casos para algunos clientes- Datos originales
- El que está transformado
- El que está invertido (recuperado)

In [ ]:
indices = df.sample(3).index

df_pn.loc[indices]

Seguramente puedes ver que algunos valores no son exactamente iguales a los de los datos originales. ¿Cuál podría ser la razón de ello?

In [ ]:
pd.DataFrame(X_ofus, columns = df_pn.columns).loc[indices]

In [ ]:
pd.DataFrame(X_ofus.dot(np.linalg.inv(P)), columns = df_pn.columns).loc[indices].round().abs().astype('int64')

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Los calculos son correctos, usaste bien numpy para esta sección.
</div>

## Prueba de que la ofuscación de datos puede funcionar con regresión lineal

En este proyecto la tarea de regresión se ha resuelto con la regresión lineal. Tu siguiente tarea es demostrar _analytically_ que el método de ofuscación no afectará a la regresión lineal en términos de valores predichos, es decir, que sus valores seguirán siendo los mismos. ¿Lo puedes creer? Pues no hace falta que lo creas, ¡tienes que que demostrarlo!

Entonces, los datos están ofuscados y ahora tenemos $X \times P$ en lugar de tener solo $X$. En consecuencia, hay otros pesos $w_P$ como
$$
w = (X^T X)^{-1} X^T y \quad \Rightarrow \quad w_P = [(XP)^T XP]^{-1} (XP)^T y
$$

¿Cómo se relacionarían $w$ y $w_P$ si simplificáramos la fórmula de $w_P$ anterior? 

¿Cuáles serían los valores predichos con $w_P$? 

¿Qué significa esto para la calidad de la regresión lineal si esta se mide mediante la RECM?
Revisa el Apéndice B Propiedades de las matrices al final del cuaderno. ¡Allí encontrarás fórmulas muy útiles!

No es necesario escribir código en esta sección, basta con una explicación analítica.

**Respuesta**

$$ w_P = P^{-1} w $$

**Prueba analítica**

$$
w = (X^T X)^{-1} X^T y \quad \Rightarrow \quad w_P = [(XP)^T XP]^{-1} (XP)^T y
$$
Agrupando en $w_P$
$$
w = (X^T)^{-1} X^{-1} X^T y \quad \Rightarrow \quad w_P = [(X^T P^T) (X P)]^{-1} X^T P^T y
$$

Agrupando y aplicando la ley de la identidad
$$
w_P = [(X^T P^T) (X P)]^{-1} X^T P^T y = (X^T X P)^{-1} (P^T)^{-1} P^T X^T y
$$

Agrupando nuevamente
$$
w_P = ((X^T X)(P))^{-1} (P^T)^{-1} P^T X^T y
$$

Reemplazando por la matriz de identidad
$$
w_P = (P)^{-1} (X^T X)^{-1} X^T y
$$

Ya que $$ w = (X^T X)^{-1} X^T y $$

Entonces 
$$ w_P = P^{-1}w $$

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

¡Muy bien! La demostración matemática es correcta.
</div>

## Prueba de regresión lineal con ofuscación de datos

Ahora, probemos que la regresión lineal pueda funcionar, en términos computacionales, con la transformación de ofuscación elegida.
Construye un procedimiento o una clase que ejecute la regresión lineal opcionalmente con la ofuscación. Puedes usar una implementación de regresión lineal de scikit-learn o tu propia implementación.
Ejecuta la regresión lineal para los datos originales y los ofuscados, compara los valores predichos y los valores de las métricas RMSE y $R^2$. ¿Hay alguna diferencia?

**Procedimiento**

- Crea una matriz cuadrada $P$ de números aleatorios.- Comprueba que sea invertible. Si no lo es, repite el primer paso hasta obtener una matriz invertible.- <¡ tu comentario aquí !>
- Utiliza $XP$ como la nueva matriz de características

In [ ]:
class MyLinearRegression:
    
    def fit(self, X, y, P=None):
        X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
        y = y
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        if P is not None:
            self.w = np.dot(np.linalg.inv(P), self.w)

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [ ]:
lr = MyLinearRegression()

lr.fit(X_train, y_train)

y_test_pred = lr.predict(X_test)
print(lr.w)
eval_regressor(y_test, y_test_pred)

In [ ]:
X = df[['age', 'gender', 'income', 'family_members']].to_numpy()
y = df['insurance_benefits'].to_numpy()

rng = np.random.default_rng(seed=42)
P = rng.random(size=(X.shape[1], X.shape[1]))

X_ofuscado = np.dot(X, P)

X_train, X_test, y_train, y_test = train_test_split(X_ofuscado, y, test_size=0.3, random_state=12345)

lr = MyLinearRegression()

lr.fit(X_train, y_train)
print(lr.w)

y_test_pred = lr.predict(X_test)

eval_regressor(y_test, y_test_pred)

Utilizando la ofuscación de los datos en el conjunto de datos de las características obtenemos los mismos valores de R2 y el error cuadrado que utilizando los datos sin ofuscar, por lo que podemos concluir que nuestra ofuscación es correcta y el modelo hace las predicciones de forma adecuada.

<div class="alert alert-block alert-danger">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Aquí estás calculando una predicción sobre X_test pero X_test no tuvo la misma ofuscación que X_train, esto se puede corregir de dos maneras:
    
1. Primero multiplica X por P antes de dividir en train y test.
    
2. Multiplica X_test por P.
    
Como consejo después de multiplicar por P cambia el nombre de la variable para diferenciar entre los datos ofuscados y los originales, es decir define las variables de la siguiente manera:
    
```
X_ofuscados_train = np.dot(X_train, P)    
```

</div>

<div class="alert alert-block alert-danger">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

También falta incluir un comentario sobre los resultados de esta sección.

</div>

# Conclusiones

En este proyecto se probo que las distancias euclidiana y manhattan nos arrojan valores muy similares por lo que continuamos evaluando con las distancias euclidianas.

Más adelante se separaron los conjuntos de datos y se evaluaron los Kneighbors a diferentes probabilidades para encontrar los valores F1, los conjuntos de datos escalados mostraron una probabilidad casi uniforme, y los valores no escalados disminuye el valor F1 conforme se incrementa la probabilidad.

Con la tarea 3 se creo una función de regresión linea que pudiera predecir si un cliente puede recibir los beneficios del seguro, por lo que usamos los conjuntos de datos de caractetísticas y objetivos separados en entrenamiento y prueba. Los resultados con los conjuntos escalados y sin escalara son muy parecidos por lo que comprobamos que nuestro modelo funciona correctamente.

En la última sección probamos como ofuscar los datos de nuestra matriz de datos para poder esconder la información confidencial de los usuarios. Esto funciono utilizando una matriz cuadrada para poder invertir, utilizando las propiedades de las matrices logramos los resultados esperados ofuscando los datos y regresando a su valor inicial.

El algebra lineal nos ayuda a poder predecir y ofuscar datos sensibles con los que estemos trabajando.

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Buenas conclusiones haciendo referencia a todos los puntos del proyecto.

</div>

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter.

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas están ordenadas de acuerdo con la lógica y el orden de ejecución
- [ ]  Se ha realizado la tarea 1
    - [ ]  Está presente el procedimiento que puede devolver k clientes similares para un cliente determinado
    - [ ]  Se probó el procedimiento para las cuatro combinaciones propuestas    - [ ]  Se respondieron las preguntas sobre la escala/distancia- [ ]  Se ha realizado la tarea 2
    - [ ]  Se construyó y probó el modelo de clasificación aleatoria para todos los niveles de probabilidad    - [ ]  Se construyó y probó el modelo de clasificación kNN tanto para los datos originales como para los escalados. Se calculó la métrica F1.- [ ]  Se ha realizado la tarea 3
    - [ ]  Se implementó la solución de regresión lineal mediante operaciones matriciales    - [ ]  Se calculó la RECM para la solución implementada- [ ]  Se ha realizado la tarea 4
    - [ ]  Se ofuscaron los datos mediante una matriz aleatoria e invertible P    - [ ]  Se recuperaron los datos ofuscados y se han mostrado algunos ejemplos    - [ ]  Se proporcionó la prueba analítica de que la transformación no afecta a la RECM    - [ ]  Se proporcionó la prueba computacional de que la transformación no afecta a la RECM- [ ]  Se han sacado conclusiones

# Apéndices

## Apéndice A: Escribir fórmulas en los cuadernos de Jupyter

Puedes escribir fórmulas en tu Jupyter Notebook utilizando un lenguaje de marcado proporcionado por un sistema de publicación de alta calidad llamado $\LaTeX$ (se pronuncia como "Lah-tech"). Las fórmulas se verán como las de los libros de texto.

Para incorporar una fórmula a un texto, pon el signo de dólar (\\$) antes y después del texto de la fórmula, por ejemplo: $\frac{1}{2} \times \frac{3}{2} = \frac{3}{4}$ or $y = x^2, x \ge 1$.

Si una fórmula debe estar en el mismo párrafo, pon el doble signo de dólar (\\$\\$) antes y después del texto de la fórmula, por ejemplo:
$$
\bar{x} = \frac{1}{n}\sum_{i=1}^{n} x_i.
$$

El lenguaje de marcado de [LaTeX](https://es.wikipedia.org/wiki/LaTeX) es muy popular entre las personas que utilizan fórmulas en sus artículos, libros y textos. Puede resultar complicado, pero sus fundamentos son sencillos. Consulta esta [ficha de ayuda](http://tug.ctan.org/info/undergradmath/undergradmath.pdf) (materiales en inglés) de dos páginas para aprender a componer las fórmulas más comunes.

## Apéndice B: Propiedades de las matrices

Las matrices tienen muchas propiedades en cuanto al álgebra lineal. Aquí se enumeran algunas de ellas que pueden ayudarte a la hora de realizar la prueba analítica de este proyecto.

<table>
<tr>
<td>Distributividad</td><td>$A(B+C)=AB+AC$</td>
</tr>
<tr>
<td>No conmutatividad</td><td>$AB \neq BA$</td>
</tr>
<tr>
<td>Propiedad asociativa de la multiplicación</td><td>$(AB)C = A(BC)$</td>
</tr>
<tr>
<td>Propiedad de identidad multiplicativa</td><td>$IA = AI = A$</td>
</tr>
<tr>
<td></td><td>$A^{-1}A = AA^{-1} = I$
</td>
</tr>    
<tr>
<td></td><td>$(AB)^{-1} = B^{-1}A^{-1}$</td>
</tr>    
<tr>
<td>Reversibilidad de la transposición de un producto de matrices,</td><td>$(AB)^T = B^TA^T$</td>
</tr>    
</table>

<div class="alert alert-block alert-warning">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

¡Hola!
    
Te felicito por tu proyecto, está bastante completo y bien realizado. Me gusto que usaste correctamente Markdown en la demostración analítica. Ya solo quedan unos detalles para que pueda aprobar tu proyecto, los cambios necesarios los coloqué en bloques de color rojo, sé que podrás realizarlo de manera exitosa. También realicé unos comentarios opcionales en bloques de color amarillo. Mucho éxito.

</div>

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Muy bien, gracias por atender a los comentarios.

</div>